In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json
import requests
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
import time

In [2]:
cid = '3cb80dac11c441e197c8b59d8106871d'
secret = 'abed4eef40ef4611bb734e5be915175c'

auth_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)

sp = spotipy.Spotify(auth_manager = auth_manager)
sp.trace = False

## 2. Getting Track Features

In [3]:
def getTrackFeatures(id):
    
    track_info = sp.track(id)
    features_info = sp.audio_features(id)

    track_name = track_info['name']
    album = track_info['album']['name']
    artist = track_info['album']['artists'][0]['name']
    release_date = track_info['album']['release_date']
    length = track_info['duration_ms']
    popularity = track_info['popularity']
    
    danceability = features_info[0]['danceability']
    energy = features_info[0]['energy']
    key = features_info[0]['key']
    loudness = features_info[0]['loudness']
    mode = features_info[0]['mode']
    speechiness = features_info[0]['speechiness']
    acousticness = features_info[0]['acousticness']
    instrumentalness = features_info[0]['instrumentalness']
    liveness = features_info[0]['liveness']
    valence = features_info[0]['valence']
    tempo = features_info[0]['tempo']
    time_signature = features_info[0]['time_signature']

    track_data = [track_name, album, artist, release_date, length, popularity, danceability, energy, 
                  key, loudness, mode, speechiness, acousticness, instrumentalness, liveness, 
                  valence, tempo, time_signature]
    
    time.sleep(0.1)

    return track_data

In [8]:
id = '3K0FE7QAh4R4Gi2jKWIPoP'
getTrackFeatures(id)

['Dvořák: From the Bohemian Forest, Op. 68, B. 182: No. 5, Silent Woods',
 'Dvorák: Cello Concerto',
 'Jacqueline du Pré',
 '1971',
 406957,
 30,
 0.123,
 0.145,
 1,
 -20.277,
 1,
 0.0458,
 0.959,
 0.869,
 0.115,
 0.0364,
 95.369,
 5]

## 1. Getting Track IDs From Playlists

In [5]:
def getTrackIDs(user, playlist_id):
    
    track_ids = []
    playlist = sp.user_playlist(user, playlist_id)
    for item in playlist['tracks']['items']:
        track = item['track']
        track_ids.append(track['id'])
    return track_ids

When playlist has more than 100 tracks:

In [5]:
def get_playlist_tracks(username,playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    track_ids = []
    
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    for item in tracks:
        track = item['track']
        track_ids.append(track['id'])
        
    return track_ids

In [6]:
track_ids = get_playlist_tracks('caroshima', 'spotify:playlist:0UhnBVrbtoGacKvWndLGWN')

In [7]:
track_ids

['3K0FE7QAh4R4Gi2jKWIPoP',
 '1GSKdr9KifUNGGLDfZ6hqh',
 '6SSpOjYkWOpJGUet6MV0Fp',
 '26z1HDhXLVaPoUxMTn20eI',
 '0EHJOhOakEiobXo81vegdy',
 '2IvnDjlgH8yy70Nz99UdcJ',
 '43bSKCMWHIuBsUVmQcDX1J',
 '6WwkffVIzWY8U5hYWn7u7B',
 '0jOnZhF75V68VsBObWx2XO',
 '76bzcAMft1MGxK5BCjmA0T',
 '5FauIrURBh6U8fWJAffI9P',
 '6RTBBi9D18PYni0pt8jSMW',
 '7wu4CDoKfHqbaJCNO2qvza',
 '7a2WqSkDlu1jqj8H1DT7og',
 '2PN5NyKKkEHU578xgfVzc3',
 '4znP6MIqYAC4fxTmNNUo7w',
 '3YqtnB10vWRQyQgE4ql47H',
 '2sTP6ghDk046UnBZqsLyyO',
 '0aDl9YSxDukKL8QKVJ2pBQ',
 '1CUbGkpZnn6qfhEtfFSXvz',
 '0EjtZmOLT6WLShJRHCFWVU',
 '6QlkHjQmo2YncQN5MQXgPZ',
 '4Ce66JznW8QbeyTdSzdGwR',
 '5iuXAcJhY1ElMthMHwl9Yp',
 '0Haa5GeQyJJ9SjSqbqcXsT',
 '1PLbGCU1s3imPfz7FZ4NI0',
 '3I1LsXYKMeWbdgJgADH8yY',
 '7sxzjre9WeKfL9g74OqmNA',
 '05d3KM9JTyNnnWIjo8PKvf',
 '7oEUdhR65vbbkK2LEuNenT',
 '2KSSbWLAWSzaso01DEP32W',
 '0GN3fXUdsTHeUg50xfDS0V',
 '7kLg5Lqoz0rueYMD3SXFJx',
 '5xK2mtKLpVu4soKn2kHEZh',
 '4yY8JqTOQyi7K4O1QcQtBG',
 '0q21FNwES2bbtcduB6kjEU',
 '0WIbzDVEpmOyBnqqdtqIL9',
 

### Trying to extract track ids from Spotify search (failed)

In [87]:
#response = requests.get('https://api.spotify.com/v1/search?query=genre%3Ajazz&type=track&offset=0&limit=50')
#response.status_code
#soup = BeautifulSoup(response.text, 'lxml')

def search_func(q):
    track_id = []
    results = sp.search(q, limit=50, offset=0, type='track', market=None)
    results = results["tracks"]["items"]
    #for item in results:
        #track = item['tracks']
        #track_id.append(tracks['id'])
    return results

In [88]:
q = 'genre:jazz'

In [ ]:
search_func(q)

## 3. List of Track IDs to DF

In [9]:
def id_to_df(track_ids):
    track_info_list = []
    
    for i in range(len(track_ids)):
        time.sleep(0.1)
        try:
            track_info = getTrackFeatures(track_ids[i])
            track_info_list.append(track_info)
        except:
            pass
        
    df =  pd.DataFrame(track_info_list, columns = ['track_name', 'album', 'artist', 'release_date', 'length', 'popularity', 
          'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness','acousticness', 
          'instrumentalness', 'liveness', 'valence', 'tempo','time_signature'])
    
    return df

In [10]:
testingmodel_df = id_to_df(track_ids)

In [11]:
testingmodel_df

,track_name,album,artist,release_date,length,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,"Dvořák: From the Bohemian Forest, Op. 68, B. 1...",Dvorák: Cello Concerto,Jacqueline du Pré,1971,406957,30,0.123,0.14500,1,-20.277,1,0.0458,0.959000,0.869000,0.1150,0.0364,95.369,5
1,"Suite bergamasque, L.75: 3. Clair de lune",The Darjeeling Limited,Various Artists,2007-01-01,302960,31,0.381,0.02970,1,-24.149,1,0.0447,0.992000,0.951000,0.0975,0.0398,64.672,4
2,Le Jardin Féerique from Ma Mère L'oye,Call Me by Your Name (Original Motion Picture ...,Various Artists,2017-11-03,182158,37,0.532,0.01100,0,-19.987,1,0.0822,0.988000,0.053000,0.0550,0.0714,57.178,4
3,"Zion Hört Die Wächter Singen (From ""Cantata Wa...",Call Me by Your Name (Original Motion Picture ...,Various Artists,2017-11-03,310369,34,0.430,0.09900,3,-18.795,1,0.0408,0.991000,0.916000,0.1030,0.3950,121.643,4
4,"Dawn - From ""Pride & Prejudice"" Soundtrack",Pride and Prejudice OST,Jean-Yves Thibaudet,2005-01-01,158600,56,0.332,0.05250,0,-24.100,1,0.0388,0.991000,0.891000,0.0683,0.0970,78.853,3
5,Agape,If Beale Street Could Talk (Original Motion Pi...,Nicholas Britell,2018-11-09,175752,63,0.153,0.16200,10,-19.610,1,0.0429,0.884000,0.943000,0.1110,0.0584,68.734,3
6,"Rêverie, L. 68: Rêverie",Debussy Piano Edition,Claude Debussy,2012-01-01,257000,59,0.296,0.00896,7,-31.948,0,0.0394,0.994000,0.910000,0.0815,0.0673,69.440,3
7,"Allegro appassionato, Op. 43",Saint-Saëns: Cello Concerto No. 1; Allegro App...,Julian Lloyd Webber,1991-04-01,206653,8,0.298,0.12600,4,-23.095,0,0.0736,0.945000,0.788000,0.1320,0.2340,137.721,4
8,"The Carnival of the Animals, R. 125: XIII. The...",Songs from the Arc of Life,Yo-Yo Ma,2015-09-18,174266,68,0.169,0.01610,9,-31.034,0,0.0442,0.986000,0.860000,0.0805,0.0573,128.143,3
9,"Elgar: Cello Concerto in E Minor, Op. 85: I. A...",Dvorák & Elgar Cello Concertos,Jacqueline du Pré,1995,480240,34,0.133,0.10400,4,-22.882,0,0.0443,0.950000,0.233000,0.2790,0.0451,82.118,4


In [12]:
testingmodel_df.to_pickle("testingmodel.pkl")

In [13]:
testingmodel_df.to_csv("testingmodel.csv")

## 4. DF To SQL

In [114]:
from sqlalchemy import create_engine
  
engine = create_engine('sqlite://')
  
practice_df.to_sql('practicing_df_to_sql', con = engine)

In [116]:
query = 'SELECT * FROM practicing_df_to_sql;'
        
pd.read_sql(query, engine)

,index,track_name,album,artist,release_date,length,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,0,Mad Lucas,Last Splash,The Breeders,1993-08-30,276866,0,0.251,0.2510,2,-19.192,1,0.0559,0.22500,0.761000,0.2000,0.1770,90.527,4
1,1,Range Life,"Crooked Rain, Crooked Rain: LA's Desert Origins",Pavement,1994,294933,6,0.555,0.7240,9,-9.429,1,0.0288,0.04450,0.000014,0.1860,0.7240,112.812,4
2,2,Angeles,Either/Or,Elliott Smith,1997-02-25,176826,53,0.530,0.0993,4,-23.548,0,0.0549,0.86000,0.200000,0.1530,0.5880,118.139,4
3,3,Sugar for the Pill,Slowdive,Slowdive,2017-05-05,270918,60,0.533,0.5450,7,-7.681,1,0.0243,0.20700,0.797000,0.1130,0.4190,98.014,4
4,4,Suffering Jukebox,"Lookout Mountain, Lookout Sea",Silver Jews,2008-06-17,261226,0,0.475,0.5050,7,-10.859,1,0.0361,0.22600,0.481000,0.0896,0.3100,127.784,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,116,Lo Boob Oscillator,Lo Boob Oscillator,Stereolab,1995,396637,50,0.358,0.8690,4,-6.076,1,0.0449,0.00241,0.102000,0.1290,0.7970,123.670,4
117,117,Avant Garde M.O.R.,The Groop Played Space Age Batchelor Pad Music,Stereolab,1993-03-22,249560,0,0.512,0.3280,4,-17.515,1,0.0291,0.58100,0.940000,0.0998,0.3010,86.882,4
118,118,Emperor Tomato Ketchup,Emperor Tomato Ketchup,Stereolab,1996-03-18,277333,18,0.687,0.9100,1,-10.968,0,0.0528,0.04380,0.007340,0.1840,0.7660,151.052,4
119,119,Changer,Stunning Debut Album,Stereolab,1991,292557,8,0.341,0.7230,7,-4.957,1,0.0302,0.00904,0.878000,0.0849,0.3740,107.356,4
